In [7]:
from roboflow import Roboflow
rf = Roboflow(api_key="0F7R4oSupQ5U1ctpBlrv")
project = rf.workspace("recycle-sx0ka").project("recycle-trnfc")
dataset = project.version(22).download("yolov8")

loading Roboflow workspace...
loading Roboflow project...
[WARNING] we noticed you are downloading a `yolov8` datasets but you don't have `ultralytics` installed. Roboflow `.deploy` supports only models trained with `ultralytics==8.0.196`, to intall it `pip install ultralytics==8.0.196`.



Extracting Dataset Version Zip to recycle-22 in yolov8:: 100%|██████████| 33972/33972 [00:15<00:00, 2159.32it/s]


In [14]:
def get_box_annotation(class_name, center_x, center_y, width, height):
    box = dict()
    box["displayName"] = class_name
    box["xMin"] = round(center_x - width / 2, 5)
    box["xMax"] = round(center_x + width / 2, 5)
    box["yMin"] = round(center_y - height / 2, 5)
    box["yMax"] = round(center_y + height / 2, 5)
    return box

In [15]:
class_traductor = {0: "Can",
                   1: "Glass",
                   2: "Plastic",
                   3: "glass"}

In [16]:
folders = [("test", "test"),
           ("train", "training"),
           ("valid", "validation")]

In [18]:
import os

json_list = []

for folder, vertex_type in folders:
    root = "recycle-22"
    files_in_directory = os.listdir(f"{root}/{folder}/images/")
    for file in files_in_directory:
        vertex_dic = dict()
        img = file
        label_name = ".".join(img.split(".")[:-1]) + ".txt"
        label = f"{root}/{folder}/labels/{label_name}"
        
        vertex_dic["imageGcsUri"] = f"gs://cloud-ai-platform-4f807e4a-a411-472d-8205-9769f14f21ef/cv/{img}"
        
        box_annotations = []
        with open(label, 'r') as file_label:
            for line in file_label:
                label_info = line.strip().split(" ")
                box_annotations.append(get_box_annotation(
                    class_traductor[ int(label_info[0]) ], 
                    float(label_info[1]),
                    float(label_info[2]),
                    float(label_info[3]), 
                    float(label_info[4])
            ))
        vertex_dic["boundingBoxAnnotations"] = box_annotations
        vertex_dic["dataItemResourceLabels"] = {"aiplatform.googleapis.com/ml_use": vertex_type}
        
        json_list.append(vertex_dic)
        

In [19]:
len(json_list)

1499

In [20]:
import json
with open("img_labels_recycle.jsonl", 'w') as file:
    for entry in json_list:
        json_line = json.dumps(entry)
        file.write(json_line + '\n')